Setting up a Convolutional Neural Network for image classification with Keras:

Used dataset: Traffic signs: kaggle.com/valentynsichkar/traffic-signs-preprocessed



In [4]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import keras

Using TensorFlow backend.


## Data

In [23]:
#load the data
data = pickle.load(open("data/data0.pickle","rb"))

In [24]:
# the data is a dictionary with the following keys:
# The dataset is already nicely prepared, therefore, no more preparation is needed. Even the splits
# are already done!
data.keys()

dict_keys(['x_test', 'y_validation', 'x_validation', 'labels', 'x_train', 'y_test', 'y_train'])

In [25]:
# extract the datasets
x_test = data["x_test"]
y_validation = data["y_validation"]
x_validation = data["x_validation"]
labels = data["labels"]
x_train = data["x_train"]
y_test = data["y_test"]
y_train = data["y_train"]

In [26]:
# delete the now obsolete original dataset
del(data)

In [32]:
# the labels are not yet in a one-hot encoding 
display(y_test[1])

# the labels are integers - therefore, the keras function to_categorical can be used to encode it
from keras.utils import to_categorical

# transformation:
def ohe(labs):
    ohe_labs = to_categorical(labs)
    return ohe_labs
y_test = ohe(y_test)
display(y_test[1])

11

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [33]:
# Nice, and the same with the others:
y_validation = ohe(y_validation)
y_train = ohe(y_train)

In [37]:
# the shape of the training data
x_train.shape

(86989, 3, 32, 32)

In [46]:
x_train = np.transpose(x_train/255.,[0,2,3,1])
x_test = np.transpose(x_test/255.,[0,2,3,1])
x_validation = np.transpose(x_validation/255.,[0,2,3,1])

In [39]:
# total number of classes:
len(y_train[1])

43

## Model building

In [47]:
# Import the necessary components from Keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

# Initialize the model object
model = Sequential()

# Add a convolutional layer
model.add(Conv2D(10, kernel_size=3, activation='relu', 
               input_shape=(32,32,3)))

# Flatten the output of the convolutional layer
model.add(Flatten())
# Add an output layer for the categories
model.add(Dense(43, activation='softmax'))

In [48]:
# Compile the model 
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Fit the model on a training set
model.fit(x_train, y_train, 
          validation_split=0.2, 
          epochs=3, batch_size=100)

Train on 69591 samples, validate on 17398 samples
Epoch 1/3
69591/69591 [==============================] - 27s 386us/step - loss: 1.0152 - accuracy: 0.7632 - val_loss: 0.4729 - val_accuracy: 0.9002
Epoch 2/3
69591/69591 [==============================] - 27s 382us/step - loss: 0.3329 - accuracy: 0.9338 - val_loss: 0.3077 - val_accuracy: 0.9402
Epoch 3/3
69591/69591 [==============================] - 26s 377us/step - loss: 0.2038 - accuracy: 0.9602 - val_loss: 0.2446 - val_accuracy: 0.9549


In [49]:
# evaluate the model:
model.evaluate(x_validation,y_validation)

4410/4410 [==============================] - 1s 157us/step


[0.6844561925956181, 0.856916069984436]